In [0]:
# connect to Google Drive 
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# install pulp for optimization problem
!pip install pulp

     |████████████████████████████████| 13.6MB 4.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [0]:
import pandas as pd
# for optimization
from pulp import *

In [0]:
downloaded = drive.CreateFile({'id': '1F72tiSyyEqMfbqNyLwcV3S02yNfEpfXJ'}) 
downloaded.GetContentFile('diet.xls')  
data = pd.read_excel('diet.xls') 

In [0]:
# remove last 3 rows because are irrelevant to the problem
data = data.iloc[:-3]


In [6]:
# check if the value are removed form the dataset
data.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [0]:
# create optimization problem
problem = LpProblem("Diet Problem", LpMinimize)

In [0]:
# create list for each variable
food = list(data['Foods'])
price = dict(zip(food, data['Price/ Serving']))
serving_size = dict(zip(food, data['Serving Size']))
calories = dict(zip(food, data['Calories']))
cholesterol = dict(zip(food, data['Cholesterol mg']))
fat = dict(zip(food, data['Total_Fat g']))
sodium = dict(zip(food, data['Sodium mg']))
carbs = dict(zip(food, data['Carbohydrates g']))
fiber = dict(zip(food, data['Dietary_Fiber g']))
protein = dict(zip(food, data['Protein g']))
vit_a = dict(zip(food, data['Vit_A IU']))
vit_c = dict(zip(food, data['Vit_C IU']))
calcium = dict(zip(food, data['Calcium mg']))
iron = dict(zip(food, data['Iron mg']))

In [0]:
# create variables
# lower bound = 0 because we can't have negative quantities
food_variables = LpVariable.dicts("Food", food, lowBound = 0, cat = 'Integer')

In [0]:
# objective function
problem += lpSum([price[i] * food_variables[i] for i in food])

In [0]:
# constraints
# we should have at least a minimum amount of calories in the diet
problem += lpSum([calories[i] * food_variables[i] for i in food]) >= 1500

# we should not have more than a certain amount of calories in the diet
problem += lpSum([calories[i] * food_variables[i] for i in food]) <= 2500

# add min and max intake for 
# cholesterol
problem += lpSum([cholesterol[i] * food_variables[i] for i in food]) >= 30
problem += lpSum([cholesterol[i] * food_variables[i] for i in food]) <= 240

# fat
problem += lpSum([fat[i] * food_variables[i] for i in food]) >= 20
problem += lpSum([fat[i] * food_variables[i] for i in food]) <= 70

# sodium
problem += lpSum([sodium[i] * food_variables[i] for i in food]) >= 800
problem += lpSum([sodium[i] * food_variables[i] for i in food]) <= 2000

# carbs
problem += lpSum([carbs[i] * food_variables[i] for i in food]) >= 130
problem += lpSum([carbs[i] * food_variables[i] for i in food]) <= 450

# fiber
problem += lpSum([fiber[i] * food_variables[i] for i in food]) >= 125
problem += lpSum([fiber[i] * food_variables[i] for i in food]) <= 250

# protein
problem += lpSum([protein[i] * food_variables[i] for i in food]) >= 60
problem += lpSum([protein[i] * food_variables[i] for i in food]) <= 100

# vit_a
problem += lpSum([vit_a[i] * food_variables[i] for i in food]) >= 1000
problem += lpSum([vit_a[i] * food_variables[i] for i in food]) <= 10000


# vit_c
problem += lpSum([vit_c[i] * food_variables[i] for i in food]) >= 400
problem += lpSum([vit_c[i] * food_variables[i] for i in food]) <= 5000


# calcium
problem += lpSum([calcium[i] * food_variables[i] for i in food]) >= 700
problem += lpSum([calcium[i] * food_variables[i] for i in food]) <= 1500


# iron
problem += lpSum([iron[i] * food_variables[i] for i in food]) >=10
problem += lpSum([iron[i] * food_variables[i] for i in food]) <= 40

In [27]:
# solve the optimization problem and print the status
problem.solve()
print("Status:", LpStatus[problem.status])

('Status:', 'Optimal')


In [28]:
# print variables values if greater than 0
for i in problem.variables():
  if i.varValue > 0:
    print(i.name + " = " + str(i.varValue))
    
print ("Total cost = " + str(round(problem.objective.value(),2)) + "$")

Food_Celery,_Raw = 41.0
Food_Kiwifruit,Raw,Fresh = 1.0
Food_Lettuce,Iceberg,Raw = 91.0
Food_Oranges = 2.0
Food_Poached_Eggs = 1.0
Food_Popcorn,Air_Popped = 14.0
Total cost = 4.89$


In [14]:
# show the problem's formulation
problem

Diet Problem:
MINIMIZE
0.23*Food_2%_Lowfat_Milk + 0.16*Food_3.3%_Fat,Whole_Milk + 0.24*Food_Apple,Raw,W_Skin + 0.16*Food_Apple_Pie + 0.16*Food_Bagels + 0.15*Food_Banana + 0.67*Food_Beanbacn_Soup,W_Watr + 0.15*Food_Bologna,Turkey + 0.05*Food_Butter,Regular + 0.31*Food_Cap'N_Crunch + 0.07*Food_Carrots,Raw + 0.04*Food_Celery,_Raw + 0.25*Food_Cheddar_Cheese + 0.28*Food_Cheerios + 0.39*Food_Chicknoodl_Soup + 0.03*Food_Chocolate_Chip_Cookies + 0.28*Food_Corn_Flks,_Kellogg'S + 0.39*Food_Couscous + 0.65*Food_Crm_Mshrm_Soup,W_Mlk + 0.27*Food_Frankfurter,_Beef + 0.16*Food_Frozen_Broccoli + 0.18*Food_Frozen_Corn + 0.32*Food_Grapes + 0.33*Food_Ham,Sliced,Extralean + 0.83*Food_Hamburger_W_Toppings + 0.31*Food_Hotdog,_Plain + 0.15*Food_Kielbasa,Prk + 0.49*Food_Kiwifruit,Raw,Fresh + 0.02*Food_Lettuce,Iceberg,Raw + 0.17*Food_Macaroni,Ckd + 0.52*Food_Malt_O_Meal,Choc + 0.99*Food_New_E_Clamchwd,W_Mlk + 0.75*Food_Neweng_Clamchwd + 0.82*Food_Oatmeal + 0.09*Food_Oatmeal_Cookies + 0.15*Food_Oranges + 0.07*F